In [2]:
import os
from datasets import load_dataset
from huggingface_hub import login

HF_LOGIN_CODE = os.getenv("HF_LOGIN_CODE")
login(HF_LOGIN_CODE)

dataset = load_dataset("aegean-ai/ai-lectures-spring-24")

In [3]:
# Persist to local folder (will create indexes, arrow files, etc.)
dataset.save_to_disk("./raw_original_video_dataset")

Saving the dataset (0/5 shards):   0%|          | 0/8 [00:00<?, ? examples/s]

In [ ]:
import tempfile
from moviepy import VideoFileClip
import os

In [ ]:
# Function to extract captions and create video chunks for a single example
def write_full_video_to_local_store(example):
    # Access the literal MP4 file (binary data) from the example
    mp4_file_data = example['mp4']  # This is the binary data of the MP4 file

    # Save the binary MP4 data to a temporary file
    with tempfile.NamedTemporaryFile(delete=False, suffix=".mp4") as temp_file:
        temp_file.write(mp4_file_data)
        temp_filename = temp_file.name  # Store the temporary file name

    # Load the video using MoviePy
    video = VideoFileClip(temp_filename)

    # Create a directory for the video chunks based on video_id
    video_dir = "full_videos"
    os.makedirs(video_dir, exist_ok=True)

    video_title_id = example['video_id']
    output_filename = os.path.join(video_dir, f"{video_title_id}_full_video.mp4")

    video.write_videofile(output_filename, codec="libx264", audio_codec="aac")

    # Clean up the temporary file after processing
    os.remove(temp_filename)

In [ ]:
dataset.map(write_full_video_to_local_store)